In [34]:
import pandas as pd
import os
import sys
import numpy as np
import sqlite3
from dataclasses import asdict

In [2]:
# To show all columns
pd.set_option('display.max_columns', None)

# If you also want to see the full content of each column (no truncation of text)
pd.set_option('display.max_colwidth', None)

In [3]:
sys.path.append(os.path.abspath('..'))

In [6]:
from technicals.indicators import Renko
from technicals.plotting_enhanced import EnhancedCandlePlot
from api.oanda_api import oanda_api as api


conn = sqlite3.connect("../state/renko_bricks.db")

In [7]:
raw_data =  pd.read_pickle("../data/GBP_NZD_M1.pkl")

In [8]:
#df_data = Renko(raw_data, brick_size=0.10)
df_data = Renko(raw_data)

In [9]:
df_raw =  df_data.iloc[-100:]
df_raw.reset_index(drop=True, inplace=True)

In [10]:
cp = EnhancedCandlePlot(df_raw)

In [12]:
cp.show_plot()

In [14]:
df_raw

,time,mid_o,mid_h,mid_l,mid_c,direction,volume,brick_duration,tick_count
0,2025-12-29 21:38:00,2.32655,2.32755,2.32655,2.32755,1,52.0,1.0,2
1,2025-12-29 21:57:00,2.32755,2.32855,2.32755,2.32855,1,109.0,1.0,2
2,2025-12-29 22:15:00,2.32855,2.32855,2.32755,2.32755,-1,213.0,1.0,2
3,2025-12-29 22:16:00,2.32755,2.32755,2.32655,2.32655,-1,75.0,1.0,2
4,2025-12-29 22:25:00,2.32655,2.32755,2.32655,2.32755,1,368.0,1.0,2
...,...,...,...,...,...,...,...,...,...
95,2025-12-31 20:08:00,2.33955,2.34055,2.33955,2.34055,1,27.0,1.0,2
96,2025-12-31 20:13:00,2.34055,2.34155,2.34055,2.34155,1,29.0,1.0,2
97,2025-12-31 21:20:00,2.34155,2.34255,2.34155,2.34255,1,39.0,1.0,2
98,2025-12-31 21:42:00,2.34255,2.34255,2.34155,2.34155,-1,20.0,1.0,2


In [15]:
pd.read_sql_query("SELECT * FROM renko_bricks", conn)

,pair,brick_index,brick_time,brick_time_ts,brick_open,brick_close,direction,ev_long,ev_short,ev_signal
0,GBP_JPY,1,2025-06-01T21:37:00+00:00,1.748814e+09,193.71200,193.61200,-1,NaN,NaN,NaN
1,GBP_JPY,2,2025-06-01T22:00:00+00:00,1.748815e+09,193.61200,193.51200,-1,NaN,NaN,NaN
2,GBP_JPY,3,2025-06-01T22:04:00+00:00,1.748815e+09,193.51200,193.61200,1,NaN,NaN,NaN
3,GBP_JPY,4,2025-06-01T22:10:00+00:00,1.748816e+09,193.61200,193.71200,1,NaN,NaN,NaN
4,GBP_JPY,5,2025-06-02T00:07:00+00:00,1.748823e+09,193.71200,193.61200,-1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
92777,AUD_NZD,1114,2025-12-31T05:22:00+00:00,1.767159e+09,1.15778,1.15878,1,NaN,NaN,NaN
92778,AUD_NZD,1115,2025-12-31T15:32:00+00:00,1.767195e+09,1.15878,1.15978,1,-1.206570,0.713956,-1.0
92779,GBP_NZD,9295,2025-12-31T21:42:00+00:00,1.767217e+09,2.34255,2.34155,-1,0.723144,-0.747518,1.0
92780,EUR_NZD,6973,2025-12-31T21:43:00+00:00,1.767217e+09,2.04139,2.04039,-1,-0.000019,-0.635146,0.0


In [36]:
bricks = pd.read_sql_query("SELECT * FROM renko_bricks", conn)

In [37]:
pair = 'GBP_NZD'.upper()

In [38]:
bricks[bricks['pair']== pair].tail(3)

,pair,brick_index,brick_time,brick_time_ts,brick_open,brick_close,direction,ev_long,ev_short,ev_signal
66675,GBP_NZD,9293,2025-12-31T20:13:00+00:00,1.767212e+09,2.34055,2.34155,1,NaN,NaN,NaN
66676,GBP_NZD,9294,2025-12-31T21:20:00+00:00,1.767216e+09,2.34155,2.34255,1,-0.260242,0.213194,-1.0
92779,GBP_NZD,9295,2025-12-31T21:42:00+00:00,1.767217e+09,2.34255,2.34155,-1,0.723144,-0.747518,1.0


In [39]:
pd.read_sql_query("SELECT * FROM renko_open_trades", conn)

,trade_id,pair,strategy_id,direction,entry_time,entry_time_ts,entry_brick_index,target_bricks,stop_bricks,tp_price,sl_price,chain_start_brick_index,last_reset_brick_index,last_seen_brick_index,last_seen_brick_time,current_step_count,chain_step_count,current_cum_reward,chain_cum_reward,reset_count,consecutive_negative_resets,last_segment_outcome
0,159266,GBP_NZD,renko_ml_v1,BUY,2025-12-31T21:42:00+00:00,1.767217e+09,9295,4,2,2.34571,2.33971,9295,9295,9295,2025-12-31T21:42:00+00:00,0,0,0,0,0,0,NEUTRAL
1,159268,CHF_JPY,renko_ml_v1,BUY,2025-12-31T21:50:00+00:00,1.767218e+09,6179,4,2,198.10900,197.50900,6179,6179,6179,2025-12-31T21:50:00+00:00,0,0,0,0,0,0,NEUTRAL


In [33]:
pd.DataFrame([asdict(t) for t in api.get_open_trades()]).sort_values(by=['unrealizedPL', 'open_time'])

,id,instrument,open_time,price,currentUnits,unrealizedPL,marginUsed,strategy_id
1,159266,GBP_NZD,2025-12-31 21:43:06.318615+00:00,2.34171,51346.0,-49.0092,3459.1287,renko_ml_v1
0,159268,CHF_JPY,2025-12-31 21:51:01.738167+00:00,197.70900,46294.0,-42.4620,2920.0206,renko_ml_v1


In [ ]:
time_query = "\
  SELECT \
    t.trade_id,\
    t.pair, \
    t.entry_time,\
    t.entry_brick_index,\
    (SELECT b.brick_index FROM renko_bricks b \
       WHERE b.pair=t.pair AND b.brick_time_ts <= t.entry_time_ts\
       ORDER BY b.brick_time_ts DESC LIMIT 1) AS prev_idx, \
    (SELECT b.brick_time FROM renko_bricks b \
       WHERE b.pair=t.pair AND b.brick_time_ts <= t.entry_time_ts \
       ORDER BY b.brick_time_ts DESC LIMIT 1) AS prev_time, \
    (SELECT b.brick_index FROM renko_bricks b \
       WHERE b.pair=t.pair AND b.brick_time_ts >= t.entry_time_ts \
       ORDER BY b.brick_time_ts ASC LIMIT 1) AS next_idx, \
    (SELECT b.brick_time FROM renko_bricks b \
       WHERE b.pair=t.pair AND b.brick_time_ts >= t.entry_time_ts \
       ORDER BY b.brick_time_ts ASC LIMIT 1) AS next_time \
       FROM renko_open_trades t;"

from datetime import datetime, timezone

server_time = api.get_server_time()
local_utc = datetime.now(timezone.utc)
local_local = datetime.now()

print("local_utc:", local_utc.isoformat())
print("local_local:", local_local.isoformat())
print("oanda_server:", server_time.isoformat() if server_time else None)
if server_time:
      print("delta_seconds:", (server_time - local_utc).total_seconds())
 